In [1]:
# Import packages (necessary for nbconvert runs)
%run packages.py
if globals()['pd']: 
    print("start script successfully loaded all packages")

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...


start script successfully loaded all packages


[nltk_data]   Package punkt is already up-to-date!


## Import additional libraries

In [2]:
import re
import itertools
import pickle
import multiprocessing as mp
from tqdm import tqdm
import os
import sys
import requests
from itertools import product
from requests.auth import HTTPBasicAuth
import json
from collections import Counter
from joblib import Parallel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
import nltk
from sklearn.neighbors import NearestNeighbors
import re
from time import time

### Import relevant UDFs

In [3]:
from src.SimMap import bag_of_words
from src.SimMap import text_col_builder
from src.SimMap import ColumnExtractor,LemmaTokenizer, DataFrameSelector, WordCutter, PatternCleaner, StopwordRemover, ColumnExtractor
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from src.udf import make_read_data_url
from src.udf import create_att_table
from src.udf import clean_raw_tables
from src.udf import make_read_data_url
from src.udf import create_att_table
from src.SimMap import cosine_sim_mp

## Set global variables

In [4]:
src_path = os.path.join(os.path.dirname(os.getcwd()), "src")
#packages = os.path.join(src_path, "packages.py")
print(src_path)

/home/ubuntu/matspec_ai/src


In [5]:
s3_path = "/S3/FILES_MT_S3/"

In [6]:
# GLOBAL VARIABLES
prod_run = True if ('prod' in os.popen('hostname').read().rstrip()[-4:]) else False # Checks if running on prod instance
hana_sys = 'q' # Can either be '0', 'q' or '1'
use_cache = True # If True, then intermediate results are stored/read on/from S3
writeback_to_hana = True

datapath = os.path.join(os.path.dirname(os.getcwd()), "data")
print(datapath)
if not os.path.exists(datapath):
    os.makedirs(datapath)

/home/ubuntu/matspec_ai/data


In [7]:
if (os.path.exists('/S3/FILES_MT_S3')):
    os.system('sudo chown -R ubuntu:iam-synced-users /S3/FILES_MT_S3')
    os.system('sudo chmod -R 777 /S3/FILES_MT_S3')

## Define odata HANA settings

In [8]:
# Get username and password for HANA
HANA_username = ("%s" % os.popen('/opt/parameter-get user').read()).rstrip('\n')
if (hana_sys == '0'):
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh0').read()).rstrip('\n')
else:
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh1').read()).rstrip('\n')

In [9]:
# Read parameters

# Create URL to CV
if (hana_sys == '1'):
    main_url = 'https://hanadatalake.siemens.com/siemens/MATSPEC/AWS/'
else:
    main_url = 'https://hanadatalake-eh' + hana_sys + '.siemens.com/siemens/MATSPEC/AWS/'

xsodata_hana_to_py = 'ODATA_CONNECTIONS.xsodata'
#source_object = '/T_IHSDOC'

## Load final ABT Table

In [89]:
df_ABT = pd.read_csv('/S3/FILES_MT_S3/df_ABT.csv', sep=';', low_memory=False)

In [90]:
df_ABT.set_index("PART_ID", inplace = True)

In [91]:
cols_cleaned = [col.strip() for col in df_ABT.columns]
df_ABT.columns = cols_cleaned

Combine all the Categories which have equal or less than 10 occurences into a group called "Others"

## Load Fit form function table

In [92]:
url = make_read_data_url(main_url, xsodata_hana_to_py, '/T_FORM_FIT_FUNCTION_ATTRIBUTE')

In [93]:
print(url)

https://hanadatalake.siemens.com/siemens/MATSPEC/AWS/ODATA_CONNECTIONS.xsodata/T_FORM_FIT_FUNCTION_ATTRIBUTE?$format=json


In [94]:
result = requests.get(url=url, auth=HTTPBasicAuth(HANA_username, HANA_password), verify=False)
df_FitFormFunction = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')

/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [95]:
df_FitFormFunction.drop(columns = ["GenID", "__metadata"], inplace = True)

In [96]:
df_FitFormFunction.head()

ATTRIBUTE_NAME DATA_SOURCE_ID  FORM  FIT  FUNCTION  \
0  1 dB Compression Point (dBm)            IHS   NaN  NaN       1.0   
1                 3dB Bandwidth             SE   NaN  NaN       1.0   
2             3dB Bandwidth (%)            IHS   NaN  NaN       1.0   
3     3dB Bandwidth Lower Range             SE   NaN  NaN       1.0   
4     3dB Bandwidth Upper Range             SE   NaN  NaN       1.0   

  ATTRIBUTE_TYPE IMPORTANCE  
0       FUNCTION          H  
1       FUNCTION          H  
2       FUNCTION          H  
3       FUNCTION          H  
4       FUNCTION          H

## Similarity Serach for form fit function and overall

## Assigning a single Categoy to each Material
Now we define the Category per each part. This is crucial since we are categories are not uniquely assigned. IN Teamcenter thee same part have multiple categories so we do not rely on TC only. Instead we do the following: 
1. Concatenate the category from IHS, taxonomy path from SE and TC category. 
2. If we see the category in this string we assign the final category to it otherwise we collect it in the other category

In [97]:
df_FitFormFunction['OVERALL'] = df_FitFormFunction.FIT.combine_first(df_FitFormFunction.FORM).combine_first(df_FitFormFunction.FUNCTION)

In [99]:
df_per_cat  = dict()
categories = df_ABT["Category"].unique()
dict_cat = dict()
att_dict = dict()
att_cols = dict()
mode = ['FIT', 'FORM', 'FUNCTION']
importance = ['H', 'M']
modes = list(product(mode, importance ))
#
#modes = ["_".join(name) for name in modes]
modes.append("OVERALL")

In [100]:
df_FitFormFunction.ATTRIBUTE_TYPE.unique()

array(['FUNCTION', 'GENERAL', 'FORM', 'FIT'], dtype=object)

In [128]:
mode = "FORM"
cols = [col for col in df_FitFormFunction.ATTRIBUTE_TYPE.unique() if mode in col]
cols[0]

'FORM'

In [101]:
for _mode in modes: 
    if _mode == "OVERALL":
        cols = set([col for col in df_FitFormFunction["ATTRIBUTE_NAME"].loc[(df_FitFormFunction["ATTRIBUTE_TYPE"] != "GENERAL")]])     
        cols.add("PART_DESCRIPTION")
    else: 
        cols = set([col for col in df_FitFormFunction["ATTRIBUTE_NAME"].loc[(df_FitFormFunction[_mode[0]] == 1) & (df_FitFormFunction["IMPORTANCE"] == _mode[1])]])
    
    att_cols[_mode] = list(cols)

In [102]:
for cat in tqdm(categories):
    att_dict_tmp = dict()
    df_per_cat[cat] = df_ABT[[col for col in att_cols["OVERALL"] if col in df_ABT.columns]].loc[df_ABT.Category == cat].dropna(axis = 1, how = "all")
    for _mode in modes:
        att_dict_tmp[_mode] = list(set([col for col in att_cols[_mode] if col in df_per_cat[cat].columns]))
    att_dict[cat] = att_dict_tmp
    #break


100%|██████████| 295/295 [32:55<00:00,  6.70s/it]


## Save dictionary to S3

In [104]:
att_dict["Nuts"]

{('FIT', 'H'): ['Outer Diameter-Min (mm)',
  'Head Height (mm)',
  'Outer Diameter-Max (mm)',
  'Thread Size',
  'Head Diameter (inch)',
  'Outer Diameter (inch)',
  'Outer Diameter (mm)',
  'Thread Direction',
  'Outer Diameter-Min (inch)',
  'Outer Diameter-Max (inch)',
  'Head Height (inch)',
  'Head Diameter (mm)'],
 ('FIT', 'M'): [],
 ('FORM', 'H'): ['Lead/Base Style'],
 ('FORM', 'M'): ['Connector Support Type', 'Fastener Type'],
 ('FUNCTION', 'H'): ['Thread Size',
  'Maximum Overall Diameter',
  'REACH Compliant',
  'Thickness-Max (mm)',
  'Operating Temperature-Max (Cel)',
  'Nominal Size (inch)',
  'Nominal Size (mm)',
  'Voltage Rating',
  'Current Rating',
  'Thickness-Max (inch)',
  'Thickness (inch)',
  'Thickness (mm)',
  'Mean Spherical Candle Power'],
 ('FUNCTION', 'M'): ['Manufacturer Series', 'Head Type', 'Head Style'],
 'OVERALL': ['Life Hours',
  'Thread Size',
  'Lead/Base Style',
  'Head Style',
  'PART_DESCRIPTION',
  'Nominal Size (mm)',
  'Outer Diameter-Min (in

In [105]:
from src.udf import save_obj

In [106]:
save_obj(df_per_cat, s3_path, "dict_df_per_category")
save_obj(att_dict, s3_path, "dict_attributes_per_category")

## Prepare data for analytics
In the following we have to create 2 sources: 
1. Build pandas df per category (based on the criteria above)
2. Build the relevant attribute per category, i.e. which columns should be considered for similarity mapping divided by form fit function and overall similarity

## Run similariy mapping: 
This is now the core of the similarity search: 
based on metric (default cosine) and based on analyzer (charachter or token level) we caclulate form, fit function and overall  similarity for all the parts within a category. The process consists of 2 stages: 
1. Build a bag of words per category
2. Calculate the k-nearest neighbor per material. 

In [139]:
from src.SimMap import cosine_sim_mp
from src.SimMap import calc_similarity

In [133]:
#mode = ["OVERALL", 'FIT', 'FORM', 'FUNCTION']
analyzer_dict = dict(zip(modes,['char_wb', 'char_wb',None , None, None, None, None]))
analyzer_dict

{('FIT', 'H'): 'char_wb',
 ('FIT', 'M'): 'char_wb',
 ('FORM', 'H'): None,
 ('FORM', 'M'): None,
 ('FUNCTION', 'H'): None,
 ('FUNCTION', 'M'): None,
 'OVERALL': None}

In [137]:
#analyzer_dict = dict(zip(mode,[None, None,None, None]))
stop_words = ['nan', 'nr', 'n/r', 'na', 'n/a']
metric = 'cosine'
#n_neighbors = 10
dict_X = dict()
results_per_cat = dict()
#category = ["led"] 

In [140]:
results = Parallel(n_jobs=-1, verbose = 1 )(delayed(calc_similarity,)(att_dict,dict_X,df_per_cat, analyzer_dict, cat, stop_words ) for cat in att_dict.keys())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 295 out of 295 | elapsed: 77.6min finished


## Consolidate result
The outcome of the above algortihm is stored into a dictionary. Transform it to the final structure needed to write results back to HANA

In [ ]:
del df_ABT
import gc 
  
# Returns the number of 
# objects it has collected 
# and deallocated 
collected = gc.collect()

In [141]:
#list_results = [df for df in results_per_cat.values()]
df_sim_map = pd.concat(results)

/opt/anaconda/envs/shared/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [142]:
df_sim_map.shape

(8891549, 7)

In [146]:
df_sim_map.FUNCTION_SIMILARITY.nunique()

30989

In [147]:
important = list(results[0].columns)
reordered = important + [c for c in df_sim_map.columns if c not in important]
df_T_SIMILARITY = df_sim_map[reordered]

In [148]:
df_T_SIMILARITY.PART_ID_2.nunique()

290388

In [149]:
round(sys.getsizeof(df_T_SIMILARITY)/1e9,2)

2.16

In [150]:
df_T_SIMILARITY.to_csv('/S3/FILES_MT_S3/df_T_SIMILARITY.csv', sep = "|", index = False)

## Write results back to HANA

In [10]:
df_T_SIMILARITY = pd.read_csv('/S3/FILES_MT_S3/df_T_SIMILARITY.csv', sep = "|")

In [11]:
df_T_SIMILARITY.shape

(8891549, 7)

In [12]:
import time
import datetime
import json
import pandas as pd
import numpy as np
import requests
import math
from requests.auth import HTTPBasicAuth
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
def write_to_hana(url, data, user, password, chunk_size=0, verbose=False):
    """
    The function writes the contents of a data frame into a corresponding Hana table.
    It maximizes the number of times to post data into the hana server by default.

    :param url: this parameter contains the url of the Odata object related to the table.
    :param data:
    :param user:
    :param password:
    :param chunk_size: amount of lines to be transferred in each step. If not changed the size is defined automatically.
    :param verbose: prints list of statistics of the transfer (optional)
    """
    from sys import getsizeof

    t0 = time.time()
    memory_threshold = 100e6  # 105 MB is the true limit.

    def upload(chunk):
        headers = {'Content-Type': 'application/json;charset=utf-8'}
        requests.post(url, chunk, auth=HTTPBasicAuth(user, password), headers=headers, verify=False)

    size = data.shape[0]

    if chunk_size > size:
        print('ERROR: Chunk size is bigger than the dataframe')
        return 'ERROR: Chunk size is bigger than the dataframe'

    if chunk_size == 0:
        
        if data.shape[0] < 1000:
            sample_size = data.shape[0]
        else:
            sample_size = 1000
            
        s = data.sample(sample_size).to_json(orient='records')  # decent sample size
        memory = size * getsizeof(s)/sample_size  # estimate of the size of the entire dataframe
        chunk_size = int(size * memory_threshold/memory)
        amount_of_transfers = size // chunk_size + 1

        if chunk_size < size:
            chunk_memory = getsizeof(data.sample(chunk_size).to_json(orient='records'))
        else:
            chunk_memory = memory

    else:
        s = data.sample(chunk_size).to_json(orient='records')
        chunk_memory = getsizeof(s)
        amount_of_transfers = size // chunk_size + 1

    for ind in range(int(amount_of_transfers)):
        data_chunk = data[ind*chunk_size:(ind+1)*chunk_size]
        data_chunk = data_chunk.to_json(orient='records')
        upload(data_chunk)
        time.sleep(0.1)

        if verbose:
            print(ind, ind*chunk_size, (ind+1)*chunk_size)

    tt = round(time.time()-t0, 4)
    if verbose:
        print(' Time spent:', tt, '\n', 'Size:', size, '\n', 'Chunksize:', chunk_size, '\n',
              'Memory per chunk:', chunk_memory, 'bytes\n','Time per record', tt/size)

In [13]:
#from src.helper_hana import write_to_hana

# Write parameters
truncate_table = "J_TRUNCATE_T_SIMILARITY.xsjs" 
target_table = 'J_INSERT_T_SIMILARITY.xsjs'

truncate_similartiy = main_url + truncate_table
write_similarity = main_url + target_table

In [14]:
df_T_SIMILARITY.head()

PART_ID_1   PART_ID_2 CATEGORY  OVERALL_SIMILARITY  FIT_SIMILARITY  \
0  00011006    00011006     Fuse              1.0000        1.000000   
1  00011006    0216016P     Fuse              0.8051        0.666667   
2  00011006  0216016MXP     Fuse              0.7603        1.000000   
3  00011006   BKGDA16AS     Fuse              0.7480        0.666667   
4  00011006  TR2GDA16AS     Fuse              0.7285        0.666667   

   FORM_SIMILARITY  FUNCTION_SIMILARITY  
0              NaN             1.000000  
1              NaN             0.583867  
2              NaN             0.413767  
3              NaN             0.685133  
4              NaN             0.428000

In [15]:
cols = df_T_SIMILARITY.columns.tolist()
new_cols = cols[:3] + cols[4:7] + [cols[3]]
new_cols

['PART_ID_1',
 'PART_ID_2',
 'CATEGORY',
 'FIT_SIMILARITY',
 'FORM_SIMILARITY',
 'FUNCTION_SIMILARITY',
 'OVERALL_SIMILARITY']

In [22]:
df.FORM_SIMILARITY.describe()

count    372097.000000
mean          0.822690
std           0.327445
min           0.000000
25%           0.796900
50%           1.000000
75%           1.000000
max           1.000000
Name: FORM_SIMILARITY, dtype: float64

In [16]:
df = df_T_SIMILARITY[new_cols]
df.head()

PART_ID_1   PART_ID_2 CATEGORY  FIT_SIMILARITY  FORM_SIMILARITY  \
0  00011006    00011006     Fuse        1.000000              NaN   
1  00011006    0216016P     Fuse        0.666667              NaN   
2  00011006  0216016MXP     Fuse        1.000000              NaN   
3  00011006   BKGDA16AS     Fuse        0.666667              NaN   
4  00011006  TR2GDA16AS     Fuse        0.666667              NaN   

   FUNCTION_SIMILARITY  OVERALL_SIMILARITY  
0             1.000000              1.0000  
1             0.583867              0.8051  
2             0.413767              0.7603  
3             0.685133              0.7480  
4             0.428000              0.7285

In [18]:
###### if writeback_to_hana:
write_to_hana(url=truncate_similartiy, data=df, user=HANA_username, password=HANA_password, chunk_size = 100000, verbose=True)
write_to_hana(write_similarity, data=df, user=HANA_username, password=HANA_password,chunk_size = 100000, verbose=True)

0 0 100000
1 100000 200000
2 200000 300000
3 300000 400000
4 400000 500000
5 500000 600000
6 600000 700000
7 700000 800000
8 800000 900000
9 900000 1000000
10 1000000 1100000
11 1100000 1200000
12 1200000 1300000
13 1300000 1400000
14 1400000 1500000
15 1500000 1600000
16 1600000 1700000
17 1700000 1800000
18 1800000 1900000
19 1900000 2000000
20 2000000 2100000
21 2100000 2200000
22 2200000 2300000
23 2300000 2400000
24 2400000 2500000
25 2500000 2600000
26 2600000 2700000
27 2700000 2800000
28 2800000 2900000
29 2900000 3000000
30 3000000 3100000
31 3100000 3200000
32 3200000 3300000
33 3300000 3400000
34 3400000 3500000
35 3500000 3600000
36 3600000 3700000
37 3700000 3800000
38 3800000 3900000
39 3900000 4000000
40 4000000 4100000
41 4100000 4200000
42 4200000 4300000
43 4300000 4400000
44 4400000 4500000
45 4500000 4600000
46 4600000 4700000
47 4700000 4800000
48 4800000 4900000
49 4900000 5000000
50 5000000 5100000
51 5100000 5200000
52 5200000 5300000
53 5300000 5400000
54 54000